# Создание нейрона с помощью NumPy

В этом ноутбуке мы:
1. Реализуем простой нейрон используя NumPy
2. Вычислим производные для обратного распространения
3. Создадим класс, который сохраняет состояния необходимые для прямого и обратного прохода


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)


## 1. Реализация простого нейрона

Нейрон выполняет следующую операцию:
$$y = \sigma(w^T x + b)$$

где:
- $x$ - входной вектор
- $w$ - вектор весов
- $b$ - смещение
- $\sigma$ - функция активации (будем использовать сигмоиду)


In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def neuron_forward(x, w, b):
    z = np.dot(w, x) + b
    output = sigmoid(z)
    return output

x = np.array([1.0, 2.0, 3.0])
w = np.array([0.5, -0.2, 0.1])
b = 0.3

output = neuron_forward(x, w, b)
print(f"Input: {x}")
print(f"Weights: {w}")
print(f"Bias: {b}")
print(f"Output: {output:.4f}")


## 2. Вычисление производных для обратного распространения

Для обратного распространения нам нужно вычислить градиенты:

**Производная сигмоиды:**
$$\frac{\partial \sigma}{\partial z} = \sigma(z) \cdot (1 - \sigma(z))$$

**Градиенты по параметрам:**
- $\frac{\partial L}{\partial w} = \frac{\partial L}{\partial y} \cdot \frac{\partial y}{\partial z} \cdot \frac{\partial z}{\partial w} = \frac{\partial L}{\partial y} \cdot \sigma'(z) \cdot x$
- $\frac{\partial L}{\partial b} = \frac{\partial L}{\partial y} \cdot \frac{\partial y}{\partial z} \cdot \frac{\partial z}{\partial b} = \frac{\partial L}{\partial y} \cdot \sigma'(z)$


In [ ]:
def sigmoid_derivative(z):
    s = sigmoid(z)
    return s * (1 - s)

def neuron_backward(x, w, b, dy):
    z = np.dot(w, x) + b
    dz = dy * sigmoid_derivative(z)
    dw = dz * x
    db = dz
    dx = dz * w
    return dw, db, dx

x = np.array([1.0, 2.0, 3.0])
w = np.array([0.5, -0.2, 0.1])
b = 0.3
dy = 1.0

dw, db, dx = neuron_backward(x, w, b, dy)
print(f"Gradient w.r.t. weights: {dw}")
print(f"Gradient w.r.t. bias: {db:.4f}")
print(f"Gradient w.r.t. input: {dx}")


## 3. Класс нейрона с управлением состоянием

Теперь создадим класс, который:
- Хранит параметры (веса и смещение)
- Кэширует промежуточные значения во время прямого прохода
- Использует кэшированные значения для эффективного обратного прохода
- Реализует обновление параметров


In [ ]:
class Neuron:
    def __init__(self, n_features):
        self.n_features = n_features
        self.w = np.random.randn(n_features) * 0.01
        self.b = 0.0
        self.cache = {}
        
    def forward(self, x):
        z = np.dot(x, self.w) + self.b
        output = sigmoid(z)
        self.cache['x'] = x
        self.cache['z'] = z
        self.cache['output'] = output
        return output
    
    def backward(self, dy):
        x = self.cache['x']
        z = self.cache['z']
        output = self.cache['output']
        dz = dy * output * (1 - output)
        
        if x.ndim == 1:
            self.dw = dz * x
            self.db = dz
            dx = dz * self.w
        else:
            batch_size = x.shape[0]
            self.dw = np.dot(x.T, dz) / batch_size
            self.db = np.mean(dz)
            dx = np.outer(dz, self.w)
        
        return dx
    
    def update_parameters(self, learning_rate):
        self.w -= learning_rate * self.dw
        self.b -= learning_rate * self.db
    
    def get_parameters(self):
        return {'w': self.w.copy(), 'b': self.b}


## 4. Пример использования класса


In [ ]:
neuron = Neuron(n_features=3)

x = np.array([1.0, 2.0, 3.0])
output = neuron.forward(x)
print(f"Output: {output:.4f}")

dy = 1.0
dx = neuron.backward(dy)
print(f"Градиент по входу: {dx}")

neuron.update_parameters(learning_rate=0.1)
print(f"Обновленные параметры: {neuron.get_parameters()}")
